In [12]:
import netCDF4
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
plt.switch_backend('agg')
#from mpl_toolkits.basemap import Basemap,addcyclic,shiftgrid
#import wrf
from wrf import getvar, to_np, CoordPair, vertcross
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from metpy.units import units
#from windspharm.standard import VectorWind
#from mpl_toolkits.basemap import Basemap, addcyclic, shiftgrid
import glob
#import pyart
import pyproj
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import sys
sys.path.append('../')
from functions_calc import geo_idx

g = pyproj.Geod(ellps='WGS84')
(az12, az21, dist) = g.inv(-116.11318, 44.16367, -115.63122, 44.590485)  

file = sorted(glob.glob("crsimNARRL6"))
file = file [::-1]
print(file)

lons,lats, Z, h, Z_cross,ah, SW, SW_cross = [],[],[],[],[],[], [], []

sp = [[33,72]]#,[31,7],[31,8]]
ep = [[126,71]]#,[127,6],[127,7]]
levs = np.arange(0,10001,40)

for i in range(0, len(file)):

    start_point = CoordPair(x = sp[i][0], y = sp[i][1])
    end_point = CoordPair(x = ep[i][0], y = ep[i][1])

    f = Dataset(file[i])
    title = file[i][-1]
    print("Getting File Variables")
    lons.append(np.array(f.variables['xlong'][:]))
    lats.append(np.array(f.variables['xlat'][:]))
    Z.append(np.array(f.variables['Zhh'][:]))
    h.append(np.array(f.variables['height'][:]))
    ah.append(np.array(f.variables['Ah'][:]))
    SW.append(np.array(f.variables['SWtot']))
    
    print(f.variables['xlat'][:][72,33])
    print(f.variables['xlat'][:][71,126])
    print(f.variables['xlong'][:][72,33])
    print(f.variables['xlong'][:][71,126])
    
    f.close()
    
    #FIRST = np.array(f.variables['Zhh'][:])
    
    #file2 = sorted(glob.glob("crsimERA5L1UP"))
    #file2 = file2 [::-1]
    #f2 = Dataset(file[i])
    
    
    #SECOND = np.array(f2.variables['Zhh'][:])
    
    #FINAL = FIRST + SECOND

    invalid = np.logical_or(ah[i]<0,ah[i]>2000)
    ah[i][invalid]=np.nan
    #For every iteration between 0 and the number of items deposited into Z
    print(Z[i].shape)
    print(len(Z[i]))
    print(Z[i][0].shape)
    print(len(Z[i][0]))
    for j in range(0,len(Z[i])):
        for k in range(0,len(Z[i][0])):
            for m in range(0,len(Z[i][0][0])):
                h_j = geo_idx(3950.0,h[i][:,k,m])
                if j < h_j:
                    Z[i][j,k,m] = Z[i][j,k,m]-2*np.nanmean(ah[i][j:h_j,k,m])*(h[i][h_j,k,m]-h[i][j,k,m])/1000.0
                if j > h_j:
                    Z[i][j,k,m] = Z[i][j,k,m]-2*np.nanmean(ah[i][h_j:j,k,m])*(h[i][j,k,m]-h[i][h_j,k,m])/1000.0

    invalid = np.logical_or(Z[i]<-40,Z[i]>20)
    Z[i][invalid]=np.nan
    Z_cross.append(vertcross(np.asarray(Z[i]), np.asarray(h[i]), np.array(levs), start_point = start_point, end_point = end_point, latlon = False, meta = False))
    SW_cross.append(vertcross(np.asarray(SW[i]), np.asarray(h[i]), np.array(levs), start_point = start_point, end_point = end_point, latlon = False, meta = False))

vmin = -40
vmax = 20
#color = pyart.graph.cm.LangRainbow12
#colorlist  = [color(i) for i in range(color.N)]
#color = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', colorlist, color.N)
bounds = np.arange(vmin,vmax+0.001,0.01)
#norm = mpl.colors.BoundaryNorm(bounds, color.N)



['crsimNARRL6']
Getting File Variables
44.210682
44.20693
-116.50195
-115.449066
(80, 141, 141)
80
(141, 141)
141


/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/cheyenne/lib/python3.7/site-packages/ipykernel_launcher.py:81: RuntimeWarning: Mean of empty slice
/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/cheyenne/lib/python3.7/site-packages/ipykernel_launcher.py:83: RuntimeWarning: Mean of empty slice
/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/cheyenne/lib/python3.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in less
/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/cheyenne/lib/python3.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in greater


In [13]:
fig, ax = plt.subplots(len(file),1,figsize=(10, 8))
for i in range(0,len(file)):

    if i == 0:
        mu = 1
    if i == 1 or i == 2:
        mu = 3
    if i == 3:
        mu = 9

    lonplot = np.linspace(lons[i][sp[i][1],sp[i][0]],lons[i][ep[0][1],ep[i][0]],len(Z_cross[i][-1,:]))
    latplot = np.linspace(lats[i][sp[i][1],sp[i][0]],lats[i][ep[0][1],ep[i][0]],len(Z_cross[i][-1,:]))
    x_ticks = np.arange(lonplot.shape[0])
    lonplot=np.round(lonplot,2)
    latplot=np.round(latplot,2)
    ax.xaxis.set_minor_locator(MultipleLocator(2000/dist*len(x_ticks)))
    ax.set_xticks([j/dist*len(x_ticks) for j in range(0,110000,10000)])

    if i == len(file)-1:
        ax.set_xticklabels(range(0,110,10))
        #ax.set_xlabel("Distance [km]")
    else:
        ax.get_xaxis().set_ticklabels([])
        
    invalid = np.logical_or(Z_cross[i]>20,Z_cross[i]<-40)
    Z_cross[i][invalid] = np.nan
    #cntrro = ax.pcolormesh(to_np(x_ticks), to_np(levs), Z_cross[i], vmin=vmin,vmax=vmax,  cmap = 'jet')
    cnt = ax.contourf(to_np(x_ticks), to_np(levs), Z_cross[i], vmin=vmin,vmax=vmax,  cmap = 'jet')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="1.5%", pad = 0.175)
    cbar = plt.colorbar(cnt, cax=cax)
    cbar.set_label("W-Band Z [dBZ]")
    cbar.set_ticks(np.arange(-40,21,10))
    cbar.set_ticklabels(np.arange(-40,21,10))

    ax.set_ylim(1000,9000)
    ax.set_yticks(np.arange(1000,9001,1000))
    ax.set_yticklabels(np.arange(1,10,1))
    ax.set_ylabel("MSL [km]")

    
#plt.text(-2200,13, "ERA5 L"+title[0], fontsize = 15)
plt.text(-2200,13, "NARR L6", fontsize = 15)
### Save Picture in right folder, rename might be required ###
plt.savefig("wbandNARRl6.png", dpi = 100)
#### Close figure to save memory ####
plt.close(fig)

/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/cheyenne/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater
/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/cheyenne/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in less


In [3]:
sp = [[72,16]]#,[31,7],[31,8]]
ep = [[71,127]]#,[127,6],[127,7]]

In [4]:
Z_cross[0].shape

(251, 112)